<a href="https://colab.research.google.com/github/nyluje/RCP211_MIDI-VAE/blob/main/RCP211_ProjetMIDI_VAE_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#QUESTION Auditeur

L'exploration du [projet réalisé en 2018](https://github.com/brunnergino/MIDI-VAE/tree/master) indique le constat suivant:
- Le code disponible sur le repository du concepteur d'origine est daté, il y a des dépendances vers des versions de librairies anciennes à différents niveaux;
- A défaut, quelques éléments, sans réaliser un reverse ingeenering des versions des librairies, semblent utilisables;
- Parmi celles-ci des fonctions semblent permettre de:
 - produire les trois jeux de données nécessaires en entrée (dit velocity roll, pitch roll et instrument roll) pour entrainer un modèle;
 - et reconstruire le résultat à partir des 3 mêmes éléments;
- Comme il est illustré ci-dessous pour un fichier MIDI donné:
 - Un échantillon audio de 30 secondes fonctionne;
 - La fonction `load_rolls` est utilisée pour en tirer des tenseurs pour l'entrainement du modèle;
 - La fonction `rolls_to_midi` est utlisée pour reconstruire la chanson, le résultat n'est pas preprésentatif de l'echantillon de 30 secondes initial.

Avez-vous un conseil pour améliorer ce résultat, afin de pouvoir tester les sorties qui seront produites par le modèle de génération de fichier MIDI?



## Installation de librairies pour manipuler les fichiers MIDI

In [1]:
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (2.1.1-2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import des librairies standards

In [2]:
import sys
import os
from IPython import display
import gdown
import pretty_midi
import fluidsynth
import numpy as np

## Installation du projet MIDI-VAE, disponible sur GIT, et modification de quelques éléments pour contourner les dépendances de librairies et garder seulement les éléments utilisés par la suite.

In [3]:
##Import
%cd /content
!rm -r ./sample_data/MIDI-VAE_PaperData
url = 'https://drive.google.com/u/0/uc?id=1OpHNPENwLn1LIQiUb1BmyYYcz1xA41b-&export=download'
output = 'dataset.zip'
gdown.download(url, output, quiet=False)
!unzip {output} -d ./sample_data
!rm -r ./sample_data/__MACOSX
!rm -r ./dataset.zip


!rm -r ./MIDI-VAE
!git clone https://github.com/brunnergino/MIDI-VAE

!sed -i 's/from matplotlib2tikz import save as tikz_save/#from matplotlib2tikz import save as tikz_save/g' ./MIDI-VAE/vae_evaluation.py
!sed -i 's/import vae_definition/#import vae_definition/g' ./MIDI-VAE/vae_evaluation.py
!sed -i 's/from vae_definition/#from vae_definition/g' ./MIDI-VAE/vae_evaluation.py
!sed -i 's/from matplotlib2tikz import save as tikz_save/#from matplotlib2tikz import save as tikz_save/g' ./MIDI-VAE/data_class.py
!sed -i '39s/^/\"\"\"/'  ./MIDI-VAE/vae_evaluation.py
!sed -i '644s/^/\"\"\"/'  ./MIDI-VAE/vae_evaluation.py
!sed -i '668s/^/\"\"\"/'  ./MIDI-VAE/vae_evaluation.py
!sed -i '2910s/^/\"\"\"/'  ./MIDI-VAE/vae_evaluation.py


/content


Downloading...
From: https://drive.google.com/u/0/uc?id=1OpHNPENwLn1LIQiUb1BmyYYcz1xA41b-&export=download
To: /content/dataset.zip
100%|██████████| 32.4M/32.4M [00:01<00:00, 24.5MB/s]


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: ./sample_data/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/013705b_.mid  
  inflating: ./sample_data/__MACOSX/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/._013705b_.mid  
  inflating: ./sample_data/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/012705b_.mid  
  inflating: ./sample_data/__MACOSX/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/._012705b_.mid  
  inflating: ./sample_data/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/011506bv.mid  
  inflating: ./sample_data/__MACOSX/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/._011506bv.mid  
  inflating: ./sample_data/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/017206ch.mid  
  inflating: ./sample_data/__MACOSX/MIDI-VAE_PaperData/Classic and Bach/barock/Bach/Bwv001- 400 Chorales/._01720

## Import des fonctions issues du projet importé

In [4]:
for i in ['/content/MIDI-VAE']:
  directory_path = os.path.abspath(os.path.join(i))
  if directory_path not in sys.path:
      sys.path.append(directory_path)

from import_midi import load_rolls as load_rolls
from midi_functions import  rolls_to_midi as rolls_to_midi
from vae_evaluation import restructure_song_to_fit_more_instruments as restructure_song_to_fit_more_instruments

/content/MIDI-VAE/midi_functions.py:109: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  hold_this_note = note in notes and (i)% SMALLEST_NOTE is not 0


## Fonction pour écouter un fichier midi (recopiée de ce [tutorial Tensorflow](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/audio/music_generation.ipynb))

In [5]:
# Sampling rate for audio playback
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

## Chargement d'un fichier midi du dataset et production d'un échantillon de 30 secondes

In [6]:
directory="/content/sample_data/MIDI-VAE_PaperData/Pop/"
songfile="ABBA_-_Waterloo.mid"
#songfile="Blur_-_Song_2.mid" #celle ci produit un vecteur vide pas lisible par la fonction display_audio suite aux opérations load_roll/rolls_to_midi dessus
sample_file=directory+songfile
pm = pretty_midi.PrettyMIDI(sample_file)
display_audio(pm, seconds=30)

## Utilisation de la fonction `load_rolls` pour produire les vecteurs utilisables pour l'entrainement du modèle

In [7]:
X, Y, Inst, Tempo, V, D=load_rolls(directory,songfile)

## En s'inspirant du code dans le fichier 'vae_evaluation.py', utilisation de la fonction `rolls_to_midi` afin de reproduire un fichier MIDI similaire à celui en entrée (le constat est que cela ne fonctionne pas, le son évoque la chanson d'origine, mais celle-ci n'est pas totalement reconstituée)

In [8]:
save_folder="./"  
filename="test"

Y_list = []
V_list = []
D_list = []
I_list = []

for y in Y:
  Y_list.extend(y)

I_list.extend(Inst)

for d in D:  
  D_list.extend(d)

for v in V:
  V_list.extend(v)

Y_list = np.asarray(Y_list)
D_list = np.asarray(D_list)
V_list = np.asarray(V_list)

Y_all_programs, all_programs, V_all_programs, D_all_programs = restructure_song_to_fit_more_instruments(Y_list, I_list, V_list, D_list)

res = rolls_to_midi(Y_all_programs,all_programs,save_folder,filename,60,V_all_programs,D_all_programs)

pm_test = pretty_midi.PrettyMIDI(save_folder +filename+".mid")

display_audio(pm_test,seconds=30)